<a href="https://colab.research.google.com/github/yjay2kim/kai/blob/main/kai_tensorflow_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools 

def load_data_keras():
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train/255.0, x_test/255.0
  return x_train, y_train, x_test, y_test

def pandas_df_to_r3_tensor(df):
  np_arr = np.array(df)
  np_arr = np_arr.reshape(99, 28, 28)
  return tf.convert_to_tensor(np_arr, dtype=tf.float32)

def load_data_pandas():
  mnist_train = pd.read_csv("/content/mnist_train_small_kai_custom.csv")
  mnist_train_features = mnist_train.copy()     
  x_train = mnist_train_features.iloc[:,1:]   # pandas dataFrame type
  y_train = mnist_train_features.iloc[:,0]
  x_train = pandas_df_to_r3_tensor(x_train)
  y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
  
  mnist_test = pd.read_csv("/content/mnist_test_kai_custom.csv")
  mnist_test_features = mnist_test.copy()
  x_test = mnist_test_features.iloc[:,1:]
  y_test = mnist_test_features.iloc[:,0]
  x_test = pandas_df_to_r3_tensor(x_test)
  y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

  x_train, x_test = x_train/255.0, x_test/255.0
  return x_train, y_train, x_test, y_test

print("TensorFlow version:", tf.__version__)

#0. Check GPU evironment
print (tf.test.gpu_device_name())

#1. Load data
#x_train, y_train, x_test, y_test = load_data_keras()
x_train, y_train, x_test, y_test = load_data_pandas()

#2. Build a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

predictions = model(x_train[:1]).numpy()
predictions

tf.nn.softmax(predictions).numpy()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
print("Done.. Model Compile")

#3. Training
model.fit(x_train, y_train, epochs=5)
print("Done.. Model fit")

#4. Evaluate
model.evaluate(x_test, y_test, verbose=2)
print("Done.. Model evaluation")

#5. Export model to SavedModel format
tf.saved_model.save(model, "/content")
print("Done.. Model export")

#6. Import model from local SavedModel format file
imported_model = tf.saved_model.load("/content")
print("Done.. Model import");

#7. Evaluate again
model.evaluate(x_test, y_test, verbose=2)
print("Done.. Model evaluation")


TensorFlow version: 2.9.2
/device:GPU:0
Done.. Model Compile
Epoch 1/5
4/4 [==============================] - 0s 4ms/step - loss: 2.3020 - accuracy: 0.1616
Epoch 2/5
4/4 [==============================] - 0s 3ms/step - loss: 1.8515 - accuracy: 0.3636
Epoch 3/5
4/4 [==============================] - 0s 4ms/step - loss: 1.5885 - accuracy: 0.5152
Epoch 4/5
4/4 [==============================] - 0s 4ms/step - loss: 1.4098 - accuracy: 0.6061
Epoch 5/5
4/4 [==============================] - 0s 3ms/step - loss: 1.1714 - accuracy: 0.6970
Done.. Model fit
4/4 - 0s - loss: 1.6676 - accuracy: 0.4949 - 102ms/epoch - 26ms/step
Done.. Model evaluation
Done.. Model export
Done.. Model import
4/4 - 0s - loss: 1.6676 - accuracy: 0.4949 - 20ms/epoch - 5ms/step
Done.. Model evaluation
